<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)  # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)  # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential()  # identity인 경우
        if stride != 1:  # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)  # (핵심) skip connection
        out = F.relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes  # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset

data_transformations = {
    "basic": transforms.Compose([
        transforms.ToTensor()
    ]),
    "crop_flip": transforms.Compose([  # 크롭 + 뒤집기
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ])
}

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["basic"])
train_aug_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["crop_flip"])
augmented_train_data = ConcatDataset((train_dataset, train_aug_dataset))

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transformations["basic"])

train_loader = torch.utils.data.DataLoader(augmented_train_data, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 90:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.1640625
Current benign train loss: 2.3044142723083496

Current batch: 100
Current benign train accuracy: 0.2421875
Current benign train loss: 1.9314301013946533

Current batch: 200
Current benign train accuracy: 0.3828125
Current benign train loss: 1.7773184776306152

Current batch: 300
Current benign train accuracy: 0.453125
Current benign train loss: 1.4190270900726318

Current batch: 400
Current benign train accuracy: 0.453125
Current benign train loss: 1.4271775484085083

Current batch: 500
Current benign train accuracy: 0.4296875
Current benign train loss: 1.4590458869934082

Current batch: 600
Current benign train accuracy: 0.625
Current benign train loss: 1.2994747161865234

Current batch: 700
Current benign train accuracy: 0.4453125
Current benign train loss: 1.4678800106048584

Total benign train accuarcy: 43.69
Total benign train loss: 1216.5600736737251

[ Test epoch: 0 ]

Test accuarcy: 29.38
Test averag


Test accuarcy: 82.41
Test average loss: 0.005359698095917702
Model Saved!

[ Train epoch: 8 ]

Current batch: 0
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2415120154619217

Current batch: 100
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2692929804325104

Current batch: 200
Current benign train accuracy: 0.9140625
Current benign train loss: 0.32681405544281006

Current batch: 300
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2870625853538513

Current batch: 400
Current benign train accuracy: 0.8984375
Current benign train loss: 0.3155900239944458

Current batch: 500
Current benign train accuracy: 0.875
Current benign train loss: 0.37069272994995117

Current batch: 600
Current benign train accuracy: 0.8671875
Current benign train loss: 0.4151807725429535

Current batch: 700
Current benign train accuracy: 0.9296875
Current benign train loss: 0.23683534562587738

Total benign train accuarcy: 90.467
Total benign tr


Total benign train accuarcy: 93.207
Total benign train loss: 154.46309987455606

[ Test epoch: 15 ]

Test accuarcy: 82.46
Test average loss: 0.006223023664951324
Model Saved!

[ Train epoch: 16 ]

Current batch: 0
Current benign train accuracy: 0.9453125
Current benign train loss: 0.15307976305484772

Current batch: 100
Current benign train accuracy: 0.90625
Current benign train loss: 0.24623870849609375

Current batch: 200
Current benign train accuracy: 0.96875
Current benign train loss: 0.10087395459413528

Current batch: 300
Current benign train accuracy: 0.859375
Current benign train loss: 0.32928726077079773

Current batch: 400
Current benign train accuracy: 0.953125
Current benign train loss: 0.15092065930366516

Current batch: 500
Current benign train accuracy: 0.921875
Current benign train loss: 0.2094358503818512

Current batch: 600
Current benign train accuracy: 0.890625
Current benign train loss: 0.3101010024547577

Current batch: 700
Current benign train accuracy: 0.921875


Current batch: 700
Current benign train accuracy: 0.9453125
Current benign train loss: 0.20453684031963348

Total benign train accuarcy: 94.228
Total benign train loss: 132.83085419237614

[ Test epoch: 23 ]

Test accuarcy: 85.15
Test average loss: 0.004889456966519355
Model Saved!

[ Train epoch: 24 ]

Current batch: 0
Current benign train accuracy: 0.90625
Current benign train loss: 0.17704574763774872

Current batch: 100
Current benign train accuracy: 0.9453125
Current benign train loss: 0.1815624237060547

Current batch: 200
Current benign train accuracy: 0.96875
Current benign train loss: 0.1111796572804451

Current batch: 300
Current benign train accuracy: 0.9453125
Current benign train loss: 0.21687288582324982

Current batch: 400
Current benign train accuracy: 0.9453125
Current benign train loss: 0.17555880546569824

Current batch: 500
Current benign train accuracy: 0.9296875
Current benign train loss: 0.2388644516468048

Current batch: 600
Current benign train accuracy: 0.953


Current batch: 600
Current benign train accuracy: 0.9296875
Current benign train loss: 0.24382135272026062

Current batch: 700
Current benign train accuracy: 0.9140625
Current benign train loss: 0.23361507058143616

Total benign train accuarcy: 94.465
Total benign train loss: 125.45093355700374

[ Test epoch: 31 ]

Test accuarcy: 88.18
Test average loss: 0.004026588612794876
Model Saved!

[ Train epoch: 32 ]

Current batch: 0
Current benign train accuracy: 0.953125
Current benign train loss: 0.12730902433395386

Current batch: 100
Current benign train accuracy: 0.953125
Current benign train loss: 0.12595847249031067

Current batch: 200
Current benign train accuracy: 0.9375
Current benign train loss: 0.1718473732471466

Current batch: 300
Current benign train accuracy: 0.9375
Current benign train loss: 0.1603129655122757

Current batch: 400
Current benign train accuracy: 0.9765625
Current benign train loss: 0.12844784557819366

Current batch: 500
Current benign train accuracy: 0.953125

...


Current batch: 300
Current benign train accuracy: 0.9921875
Current benign train loss: 0.02213037945330143

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.0015934426337480545

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.0015444152522832155

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.004262750037014484

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.002349278423935175

Total benign train accuarcy: 99.811
Total benign train loss: 5.5717940000176895

[ Test epoch: 63 ]

Test accuarcy: 94.62
Test average loss: 0.002079338104836643
Model Saved!

[ Train epoch: 64 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0025366225745528936

Current batch: 100
Current benign train accuracy: 0.984375
Current benign train loss: 0.021459326148033142

Current batch: 200
Current benign train accuracy: 0.9921875
Current be


Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.004012881778180599

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.003979517146945

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.0022677944507449865

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.002865625312551856

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.005760852247476578

Total benign train accuarcy: 99.875
Total benign train loss: 4.05023058943334

[ Test epoch: 71 ]

Test accuarcy: 94.55
Test average loss: 0.0021358543507754802
Model Saved!

[ Train epoch: 72 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0010756105184555054

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0035842726938426495

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0


Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0026913860347121954

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.004420410376042128

Current batch: 500
Current benign train accuracy: 0.9921875
Current benign train loss: 0.04990636929869652

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.00968315452337265

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.001249187858775258

Total benign train accuarcy: 99.905
Total benign train loss: 3.5945471543527674

[ Test epoch: 79 ]

Test accuarcy: 94.49
Test average loss: 0.0022194336164742708
Model Saved!

[ Train epoch: 80 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.001147982431575656

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0013045043451711535

Current batch: 200
Current benign train accuracy: 1.0
Current benign train l


Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0018820855766534805

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.006725804414600134

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.00221926043741405

Current batch: 600
Current benign train accuracy: 0.9921875
Current benign train loss: 0.009955352172255516

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.004735885187983513

Total benign train accuarcy: 99.828
Total benign train loss: 5.1386112864129245

[ Test epoch: 87 ]

Test accuarcy: 94.1
Test average loss: 0.002291753102838993
Model Saved!

[ Train epoch: 88 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.007961979135870934

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.004012645222246647

Current batch: 200
Current benign train accuracy: 1.0
Current benign train los


Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0004581136454362422

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.001648353529162705

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.000475933775305748

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.0009294230840168893

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.001681899419054389

Total benign train accuarcy: 99.972
Total benign train loss: 1.5500703225843608

[ Test epoch: 95 ]

Test accuarcy: 95.0
Test average loss: 0.0020183881592005493
Model Saved!

[ Train epoch: 96 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.001797132776118815

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0005955667002126575

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: